Use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Use the k-means clustering algorithm to complete this task. Use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

In [2]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize

import folium
import json

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# 1. Dowload and Explore the dataset

In [19]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [17]:
neighborhoods_data=newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [24]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

df=pd.DataFrame(columns=column_names)

In [29]:
for data in neighborhoods_data:
    neig_b=data['properties']['borough']
    neig_name=data['properties']['name']
    neig_latlon=data['geometry']['coordinates']
    neig_lat=neig_latlon[1]
    neig_lon=neig_latlon[0]
    
    df=df.append({'Borough':neig_b, 
                  'Neighborhood':neig_name,
                  'Latitude':neig_lat, 
                  'Longitude':neig_lon
        
    },ignore_index=True)

In [30]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [42]:
print((df['Borough'].unique()))
print(df.shape)

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']
(306, 4)


In [37]:
address='New York City, NY'
geolocator=Nominatim(user_agent='ny_explore')
location=geolocator.geocode(address)
lat=location.latitude
lon=location.longitude
print('the location of NY is {},{}'.format(lat,lon))

the location of NY is 40.7127281,-74.0060152


In [41]:
maps=folium.Map(location=[lat,lon],zoom_start=10)

for la,lo,b,n in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label='{},{}'.format(b,n)
    label=folium.Popup(label,parse_html=True)
    
    folium.CircleMarker(
    [la,lo],
    popup=label,
    radius=5,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(maps)

maps
    

To know about Manhattan more:

In [111]:
a=df[ df['Borough'] == 'Manhattan'].reset_index(drop=True)
a.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [52]:
address='Manhattan, NY'
geolocator=Nominatim(user_agent='ny_explore')
location=geolocator.geocode(address)
lat=location.latitude
lon=location.longitude
print('the location of Manhattan is {},{}'.format(lat,lon))

the location of Manhattan is 40.7896239,-73.9598939


In [60]:
maps1= folium.Map(location=[lat,lon],zoom_start=11)

for la,lo,b,n in zip(a['Latitude'],a['Longitude'],a['Borough'],a['Neighborhood']):
    label='{},{}'.format(la,lo)
    label=folium.Popup(label,parse_html=True)
    
    folium.CircleMarker(
    [la,lo],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_opacity=0.5,
    fill_color='#3186cc',
    parase_html=False).add_to(maps1)
maps1

To explore the neighbours of the first place in manhattan:

In [61]:
CLIENT_ID = 'NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N' # your Foursquare ID
CLIENT_SECRET = 'WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ' # your Foursquare Secret
ACCESS_TOKEN = 'AWCDFCLJBBPUKSG2FX5TTMHSRG2OEIPTEQ5P3CH0DBSENWWX' #access_token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N
CLIENT_SECRET:WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ


In [62]:
a.loc[0,'Neighborhood']

'Marble Hill'

In [64]:
neighborhood_latitude = a.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = a.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = a.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [70]:
url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude,VERSION,500,15)

In [71]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601d4886d650d43b2080ed9f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [72]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [381]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [382]:
# type your answer here
manhattan_venues = getNearbyVenues(names=a['Neighborhood'],latitudes=a['Latitude'],longitudes=a['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [383]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3171, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [384]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,70,70,70,70,70,70
Carnegie Hill,86,86,86,86,86,86
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,39,39,39,39,39,39
East Village,100,100,100,100,100,100


In [386]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 333 uniques categories.


# 3. Analyse each Neighborhood

In [399]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [400]:
manhattan_onehot.shape

(3171, 334)

In [401]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.00,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.00,0.000000,0.011628,0.00,0.000000,0.011628,0.000000,...,0.00,0.011628,0.000000,0.000000,0.000000,0.011628,0.034884,0.00,0.011628,0.034884
2,Central Harlem,0.000000,0.000000,0.00,0.068182,0.045455,0.00,0.000000,0.000000,0.022727,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.060000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,0.010000,0.000000
4,Chinatown,0.000000,0.000000,0.00,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000
5,Civic Center,0.000000,0.000000,0.00,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.01,0.000000,0.030000
6,Clinton,0.000000,0.000000,0.00,0.000000,0.040000,0.00,0.000000,0.000000,0.010000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.040000,0.00,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8,East Village,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.000000,0.010000,0.010000,...,0.00,0.030000,0.000000,0.000000,0.000000,0.040000,0.010000,0.00,0.000000,0.000000
9,Financial District,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.010000,0.010000


In [402]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp=manhattan_grouped[manhattan_grouped['Neighborhood']==hood].T.reset_index()
    temp=temp.iloc[1:,:]
    temp.columns=['ven','freq']
    temp['freq']=temp['freq'].astype(float).round(2)
    temp=temp.sort_values(by='freq',ascending=False).reset_index(drop=True).head(5)
    print(temp)

----Battery Park City----
             ven  freq
0    Coffee Shop  0.09
1          Hotel  0.07
2           Park  0.06
3  Memorial Site  0.04
4            Gym  0.04
----Carnegie Hill----
              ven  freq
0     Coffee Shop  0.09
1            Café  0.05
2     Yoga Studio  0.03
3     Pizza Place  0.03
4  Cosmetics Shop  0.03
----Central Harlem----
                   ven  freq
0   African Restaurant  0.07
1                  Bar  0.05
2   Chinese Restaurant  0.05
3  American Restaurant  0.05
4   Seafood Restaurant  0.05
----Chelsea----
                   ven  freq
0          Art Gallery  0.06
1          Coffee Shop  0.06
2               Bakery  0.05
3    French Restaurant  0.04
4  American Restaurant  0.03
----Chinatown----
                   ven  freq
0   Chinese Restaurant  0.08
1               Bakery  0.07
2  American Restaurant  0.04
3    Hotpot Restaurant  0.04
4         Cocktail Bar  0.04
----Civic Center----
                    ven  freq
0           Coffee Shop  0.07
1  Gym / F

In [391]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [403]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Clothing Store,Gym,Memorial Site,Food Court,Shopping Mall,Beer Garden,Plaza
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Wine Shop,Cosmetics Shop,Bookstore,Gym,Bar,Japanese Restaurant
2,Central Harlem,African Restaurant,Chinese Restaurant,French Restaurant,Gym / Fitness Center,American Restaurant,Bar,Cosmetics Shop,Seafood Restaurant,Gym,Grocery Store
3,Chelsea,Coffee Shop,Art Gallery,Bakery,French Restaurant,Ice Cream Shop,Wine Shop,Seafood Restaurant,American Restaurant,Italian Restaurant,Cupcake Shop
4,Chinatown,Chinese Restaurant,Bakery,American Restaurant,Hotpot Restaurant,Cocktail Bar,Salon / Barbershop,Dessert Shop,Spa,Optical Shop,Ice Cream Shop


# 4. Cluster Neighborhoods

In [416]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)
    
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 4, 0, 1, 1, 3, 0, 1])

In [430]:
# add clustering labels

#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_
manhattan_merged = a

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Sandwich Place,Coffee Shop,Gym,Tennis Stadium,Steakhouse,Donut Shop,Kids Store,Seafood Restaurant,Pharmacy
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,American Restaurant,Hotpot Restaurant,Cocktail Bar,Salon / Barbershop,Dessert Shop,Spa,Optical Shop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Spanish Restaurant,Grocery Store,Mobile Phone Shop,Deli / Bodega,Sandwich Place,Latin American Restaurant,Gym,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Café,Mexican Restaurant,Lounge,Restaurant,Chinese Restaurant,Deli / Bodega,Wine Bar,Pizza Place,Bakery,Caribbean Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Café,Coffee Shop,Mexican Restaurant,Deli / Bodega,Latin American Restaurant,Liquor Store,Sushi Restaurant,Cocktail Bar,Sandwich Place


In [431]:
# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [432]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,American Restaurant,Hotpot Restaurant,Cocktail Bar,Salon / Barbershop,Dessert Shop,Spa,Optical Shop,Ice Cream Shop
2,Washington Heights,Café,Bakery,Spanish Restaurant,Grocery Store,Mobile Phone Shop,Deli / Bodega,Sandwich Place,Latin American Restaurant,Gym,Tapas Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,French Restaurant,Gym / Fitness Center,American Restaurant,Bar,Cosmetics Shop,Seafood Restaurant,Gym,Grocery Store
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Deli / Bodega,Sushi Restaurant,Japanese Restaurant,Wine Shop,Diner,Park
12,Upper West Side,Bakery,Café,Bar,Wine Bar,Italian Restaurant,Indian Restaurant,Pizza Place,Seafood Restaurant,Bagel Shop,Gym / Fitness Center
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Coffee Shop,Café,Gym,Dessert Shop,Indian Restaurant,American Restaurant,Vietnamese Restaurant
19,East Village,Bar,Pizza Place,Wine Bar,Mexican Restaurant,Korean Restaurant,Vietnamese Restaurant,Salon / Barbershop,Coffee Shop,Dessert Shop,Ice Cream Shop
20,Lower East Side,Chinese Restaurant,Art Gallery,Latin American Restaurant,Bakery,Café,Coffee Shop,Japanese Restaurant,Clothing Store,Mediterranean Restaurant,Bubble Tea Shop
22,Little Italy,Bakery,Chinese Restaurant,Cocktail Bar,Café,Italian Restaurant,Hotel,Sandwich Place,Mediterranean Restaurant,Bubble Tea Shop,Salon / Barbershop
25,Manhattan Valley,Coffee Shop,Yoga Studio,Pizza Place,Mexican Restaurant,Thai Restaurant,Indian Restaurant,Bar,Cosmetics Shop,Farmers Market,Cocktail Bar


In [433]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Sandwich Place,Coffee Shop,Gym,Tennis Stadium,Steakhouse,Donut Shop,Kids Store,Seafood Restaurant,Pharmacy
8,Upper East Side,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Gym / Fitness Center,American Restaurant,Yoga Studio,Hotel,Juice Bar,Spa
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Cocktail Bar,Coffee Shop,Café,Pizza Place,Gym,Gym / Fitness Center,Burger Joint,Sporting Goods Shop
13,Lincoln Square,Plaza,Gym / Fitness Center,Café,Performing Arts Venue,Theater,Italian Restaurant,Concert Hall,Indie Movie Theater,Wine Shop,Coffee Shop
14,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,Coffee Shop,Wine Shop,Gym,American Restaurant,Cocktail Bar,Hotel,Sandwich Place
16,Murray Hill,Coffee Shop,Sandwich Place,Bar,American Restaurant,Hotel,Gym / Fitness Center,Japanese Restaurant,Burger Joint,Gym,Juice Bar
21,Tribeca,Italian Restaurant,Park,Wine Bar,American Restaurant,Spa,Café,Coffee Shop,French Restaurant,Skate Park,Scenic Lookout
24,West Village,Italian Restaurant,American Restaurant,Cocktail Bar,New American Restaurant,Wine Bar,Park,Cosmetics Shop,Coffee Shop,Theater,French Restaurant
29,Financial District,Coffee Shop,Bar,Italian Restaurant,Cocktail Bar,Park,Mexican Restaurant,Gym / Fitness Center,Pizza Place,Gym,Hotel
32,Civic Center,Coffee Shop,Gym / Fitness Center,Spa,Hotel,Cocktail Bar,French Restaurant,Yoga Studio,Park,American Restaurant,Italian Restaurant


In [434]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Boat or Ferry,Coffee Shop,Cocktail Bar,Heliport,Gym / Fitness Center,Farmers Market,Pet Service,Harbor / Marina


In [435]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Café,Mexican Restaurant,Lounge,Restaurant,Chinese Restaurant,Deli / Bodega,Wine Bar,Pizza Place,Bakery,Caribbean Restaurant
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Mexican Restaurant,Deli / Bodega,Latin American Restaurant,Liquor Store,Sushi Restaurant,Cocktail Bar,Sandwich Place
5,Manhattanville,Coffee Shop,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Deli / Bodega,Italian Restaurant,Latin American Restaurant,History Museum,Supermarket,Boutique
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Sandwich Place,New American Restaurant,Spa,Pizza Place,French Restaurant
11,Roosevelt Island,Park,Gym,Dog Run,Plaza,Soccer Field,Farmers Market,Supermarket,Metro Station,Outdoors & Recreation,School
26,Morningside Heights,Coffee Shop,Park,American Restaurant,Bookstore,Burger Joint,Café,Mediterranean Restaurant,Greek Restaurant,Seafood Restaurant,Sandwich Place
35,Turtle Bay,Coffee Shop,Italian Restaurant,Hotel,Seafood Restaurant,Deli / Bodega,Ramen Restaurant,Park,Japanese Restaurant,Sushi Restaurant,Pharmacy
36,Tudor City,Park,Café,Mexican Restaurant,Deli / Bodega,Greek Restaurant,Coffee Shop,Sushi Restaurant,Dog Run,Pizza Place,Vietnamese Restaurant


In [436]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Hotel,Coffee Shop,Clothing Store,Theater,Bookstore,Steakhouse,Bakery,Gym,Sporting Goods Shop,Spa
17,Chelsea,Coffee Shop,Art Gallery,Bakery,French Restaurant,Ice Cream Shop,Wine Shop,Seafood Restaurant,American Restaurant,Italian Restaurant,Cupcake Shop
23,Soho,Clothing Store,Italian Restaurant,Coffee Shop,Bakery,Mediterranean Restaurant,Hotel,Salon / Barbershop,Boutique,Sporting Goods Shop,French Restaurant
28,Battery Park City,Coffee Shop,Hotel,Park,Clothing Store,Gym,Memorial Site,Food Court,Shopping Mall,Beer Garden,Plaza
33,Midtown South,Korean Restaurant,Hotel,Hotel Bar,Gym / Fitness Center,Burger Joint,Coffee Shop,Café,Bakery,Japanese Restaurant,Salad Place
